# Multi-Area Network Simulation and Analysis

This Jupyter Notebook documents the workflow for simulating and analyzing a multi-area network model. The notebook is structured into several sections, each focusing on a specific part of the process, from setting up the environment to running simulations and performing analyses. Below is an overview of the key steps involved:

1. **Setting Up the Environment**: Import necessary modules and define paths and parameters. This includes importing default network, simulation, and analysis parameters, as well as setting the base path for experiments. These steps are crucial for ensuring that the subsequent steps in the workflow have access to the required configurations and data.

2. **Creating the Network**: Utilize the `NeuronNumbers` and `SynapseNumbers` classes to calculate the number of neurons and synapses in the network. The network is then created using these calculations and exported for further use.

3. **Running the Simulation**: Set up and run the simulation using the defined network and simulation parameters. The simulation results are exported and can be used for analysis.

4. **Analysis**: Perform various analyses on the simulation results, including plotting raster plots and calculating mean firing rates. The analysis results are exported for further inspection.

In [1]:
import os
import sys
import importlib.util

sys.path.append('./src/')

from helpers.snakemake import nested_dict_update, get_git_revision_hash
from default_net_params import params as net_params
from data_preprocessing.cytoarchitecture import NeuronNumbers
from data_preprocessing.connectivity import SynapseNumbers
from network import Network

In [2]:
# Import the default simulation parameters
from default_sim_params import params as sim_params
from network import networkDictFromDump
from simulation import Simulation

[INFO] [2024.10.15 22:7:54 /home/conda/feedstock_root/build_artifacts/nest-simulator_1636957195130/work/nestkernel/rng_manager.cpp:217 @ Network::create_rngs_] : Creating default RNGs
[INFO] [2024.10.15 22:7:54 /home/conda/feedstock_root/build_artifacts/nest-simulator_1636957195130/work/nestkernel/rng_manager.cpp:260 @ Network::create_grng_] : Creating new default global RNG

              -- N E S T --
  Copyright (C) 2004 The NEST Initiative

 Version: nest-2.20.2
 Built: Nov 15 2021 06:23:02

 This program is provided AS IS and comes with
 NO WARRANTY. See the file LICENSE for details.

 Problems or suggestions?
   Visit https://www.nest-simulator.org

 Type 'nest.help()' to find out more about NEST.



In [3]:
# Import the default analysis parameters
from simulation import simulationDictFromDump
from analysis import Analysis
from default_ana_params import params as ana_params

In [4]:
# Set base path
base_path = os.getcwd()
experiments_path = os.path.join(base_path, 'experiments')

### Setting Up the Environment

In this section, we set up the environment by importing necessary modules and defining paths and parameters. This includes importing default network, simulation, and analysis parameters, as well as setting the base path for experiments. These steps are crucial for ensuring that the subsequent steps in the workflow have access to the required configurations and data.

In [5]:
# Set default parameters
outpath = net_params['outpath']

# Set simulation parameters
sim_params['t_sim'] = 2000.0
sim_params['master_seed'] = 2903

# Set plotting parameters
ana_params['plotRasterArea']['low'] = 1600
ana_params['plotRasterArea']['high'] = 2000
ana_params['plotRasterArea']['fraction'] = 0.5

# Set scaling parameters
scaling_factor = 0.005
net_params['N_scaling'] = scaling_factor
net_params['K_scaling'] = scaling_factor

# Mean fullscale firing rate
net_params['fullscale_rates'] = './simulated_data/base_theory_rates.pkl'

### Creating the network

Here the network is created using the `NeuronNumbers` and `SynapseNumbers` classes. The `NeuronNumbers` class calculates the number of neurons in the network. The `SynapseNumbers` class calculates the number of synapses in the network and the synaptic weights.

In [6]:
# Create Network class
NN = NeuronNumbers(
    surface_area=net_params['surface_area'],
    **net_params['cytoarchitecture_params']
)

Dropping layer IV in caudalmiddlefrontal because it contains 3375 < 5000 neurons.
Dropping layer IV in lateraloccipital because it contains 3000 < 5000 neurons.
Dropping layer VI in parahippocampal because it contains 3200 < 5000 neurons.
Dropping layer IV in parsorbitalis because it contains 625 < 5000 neurons.


/home/shimoura/Github/humam-add-downscaling-factor/./src/data_preprocessing/cytoarchitecture.py:150: FutureWarning:iteritems is deprecated and will be removed in a future version. Use .items instead.


In [7]:
# Create Synapse class
SN = SynapseNumbers(
    NN=NN,
    **net_params['predictive_connectomic_params']
)

In [8]:
# Create Network
net = Network(NN, SN, net_params)

# Export the network
net.dump(outpath)

# Get the network hash
net_hash = net.getHash()

# To access net object key, use net.net.keys()
# Ex.: to access neuron numbers, use net.net['neuron_numbers']

### Running the simulation

In [9]:
# Read network dict
net_folder = os.path.join(outpath, net_hash)
net_dict = networkDictFromDump(net_folder)

/home/shimoura/Github/humam-add-downscaling-factor/./src/network.py:892: YAMLLoadWarning:calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
/home/shimoura/Github/humam-add-downscaling-factor/./src/network.py:897: YAMLLoadWarning:calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
/home/shimoura/Github/humam-add-downscaling-factor/./src/network.py:902: YAMLLoadWarning:calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
/home/shimoura/Github/humam-add-downscaling-factor/./src/network.py:907: YAMLLoadWarning:calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.


In [10]:
# Create Simulation class, export it and calculate the hash
sim = Simulation(sim_params, net_dict)
sim.dump(net_folder)
sim_hash = sim.getHash()

In [11]:
net.net['neuron_numbers'].sum()

17350

In [12]:
# Set output directory according to hashes, instantiate the Network
# and run the simulation
data_path = os.path.join(outpath, net_hash, sim_hash)
num_threads = int(8)
sim.setup(data_path, num_threads)
sim.simulate()

/home/shimoura/Github/humam-add-downscaling-factor/./src/simulation.py:125: FutureWarning:iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/shimoura/miniconda3/envs/humam_nest2.20.2/lib64/python3.9/site-packages/nest/lib/hl_api_helper.py:127: UserWarning:
GetNodes is deprecated and will be removed in NEST 3.0. Use             GIDCollection instead.


data directory already exists
Data will be written to /home/shimoura/Github/humam-add-downscaling-factor/out/5c92d51068c956a81d67d16610aa9dcc/e6f6894f7ce545b48d43876532c39947
Master seed: 2903 

Oct 15 22:09:27 ModelManager::clear_models_ [Info]: 
    Models will be cleared and parameters reset.

Oct 15 22:09:27 Network::create_rngs_ [Info]: 
    Deleting existing random number generators

Oct 15 22:09:27 Network::create_rngs_ [Info]: 
    Creating default RNGs

Oct 15 22:09:27 Network::create_grng_ [Info]: 
    Creating new default global RNG

Oct 15 22:09:27 ModelManager::clear_models_ [Info]: 
    Models will be cleared and parameters reset.

Oct 15 22:09:27 Network::create_rngs_ [Info]: 
    Deleting existing random number generators

Oct 15 22:09:27 Network::create_rngs_ [Info]: 
    Creating default RNGs

Oct 15 22:09:27 Network::create_grng_ [Info]: 
    Creating new default global RNG
Number of total processes: 8
Seeds for random number generators of virtual processes: [2912, 2

/home/shimoura/Github/humam-add-downscaling-factor/./src/simulation.py:249: FutureWarning:iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/shimoura/Github/humam-add-downscaling-factor/./src/simulation.py:265: FutureWarning:iteritems is deprecated and will be removed in a future version. Use .items instead.


Connected all of area bankssts, layer VI and population I on rank 0. Memory: 5464.25 MB.
Connected all of area caudalanteriorcingulate, layer II/III and population E on rank 0. Memory: 5464.25 MB.
Connected all of area caudalanteriorcingulate, layer II/III and population I on rank 0. Memory: 5464.25 MB.
Connected all of area caudalanteriorcingulate, layer V and population E on rank 0. Memory: 5464.25 MB.
Connected all of area caudalanteriorcingulate, layer V and population I on rank 0. Memory: 5464.25 MB.
Connected all of area caudalanteriorcingulate, layer VI and population E on rank 0. Memory: 5464.25 MB.
Connected all of area caudalanteriorcingulate, layer VI and population I on rank 0. Memory: 5464.25 MB.
Connected all of area caudalmiddlefrontal, layer II/III and population E on rank 0. Memory: 5464.25 MB.
Connected all of area caudalmiddlefrontal, layer II/III and population I on rank 0. Memory: 5464.25 MB.
Connected all of area caudalmiddlefrontal, layer V and population E on ra

### Analysis

In [13]:
# Read simulation dict
sim_folder = os.path.join(outpath, net_hash, sim_hash)
sim_dict = simulationDictFromDump(sim_folder)

# Create Analysis class and export it
ana = Analysis(ana_params, net_dict, sim_dict, sim_folder, base_path)
ana.dump(sim_folder)

# Do the analysis
# ana.fullAnalysis()

ana.popGids = ana._readPopGids()
ana.spikes = ana._readSpikes()
for area in net.net['area_list']:
    ana.plotRasterArea(area)

Results will be written to /home/shimoura/Github/humam-add-downscaling-factor/out/5c92d51068c956a81d67d16610aa9dcc/e6f6894f7ce545b48d43876532c39947/f24eba850dd1538ce2f9ef65d941b8b8/plots
Plots will be written to /home/shimoura/Github/humam-add-downscaling-factor/out/5c92d51068c956a81d67d16610aa9dcc/e6f6894f7ce545b48d43876532c39947/f24eba850dd1538ce2f9ef65d941b8b8/plots
_readPopGids took 0.006 s
Loading SpikeTrains from gdf


/home/shimoura/Github/humam-add-downscaling-factor/./src/simulation.py:553: YAMLLoadWarning:calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.


_readSpikes took 0.983 s


/home/shimoura/Github/humam-add-downscaling-factor/./src/analysis.py:1289: FutureWarning:iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/shimoura/Github/humam-add-downscaling-factor/./src/analysis.py:1289: FutureWarning:iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/shimoura/Github/humam-add-downscaling-factor/./src/analysis.py:1289: FutureWarning:iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/shimoura/Github/humam-add-downscaling-factor/./src/analysis.py:1289: FutureWarning:iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/shimoura/Github/humam-add-downscaling-factor/./src/analysis.py:1289: FutureWarning:iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/shimoura/Github/humam-add-downscaling-factor/./src/analysis.py:1289: FutureWarning:iteritems is deprecated and will be remov

In [14]:
ana.meanFiringRate()

meanFiringRate took 0.028 s


area                layer   pop
bankssts            II/III  E       27.544218
                            I       39.663492
                    IV      E      100.352490
                            I      131.188406
                    V       E       30.805281
                                      ...    
transversetemporal  IV      I       84.454106
                    V       E       58.795918
                            I       54.500000
                    VI      E      114.404762
                            I      201.666667
Length: 254, dtype: float64